In [58]:
# Set env var OPENAI_API_KEY or load from a .env file
import dotenv

dotenv.load_dotenv()

True

In [59]:
import os

os.environ["LANGSMITH_TRACING"] = "false"

my_test = os.getenv("LANGSMITH_TRACING")
print(my_test)

false


In [60]:
from langchain_community.document_loaders import WebBaseLoader

doc_url = "https://langchain-ai.github.io/langgraph/troubleshooting/errors/" #"https://python.langchain.com/docs/contributing/" #"https://lilianweng.github.io/posts/2023-06-23-agent/"
loader = WebBaseLoader(doc_url)
docs = loader.load()

print(docs)

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/troubleshooting/errors/', 'title': 'Troubleshooting', 'description': 'Build language agents as graphs', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\nTroubleshooting\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\nJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            \n              Troubleshooting\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n\n\n          \n  \n    \n  \n  Home\n\n        \n\n\n\n          \n  \n    \n  \n  API reference\n\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n    \n  \n    Home\n  \n\n    \n  \n\n\n\n\n\n\n\n\n            \n  \n    Home\n  \n\n    

In [64]:
from langchain_ollama import ChatOllama

#llm = ChatOllama(model="llama3.2", temperature=0)
llm = ChatOllama(model="phi4", temperature=0)

## Summarize in a single LLM call

In [65]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import  LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result = chain.invoke({"context": docs})
print(result)

The document provides a guide on troubleshooting common errors encountered while using the LangGraph platform. It includes links to various sections such as "Home," "API reference," and specific error guides like "GRAPH_RECURSION_LIMIT" and "INVALID_LICENSE." The page is part of GitHub's content, offering resources for resolving issues related to LangGraph development. Each error has a corresponding `lc_error_code` for identification. Additionally, there are references to other sections such as "Prebuilt Agents," "Adopters," and "FAQ." The document also mentions an upcoming event, the Agent AI Conference by LangChain in San Francisco.


In [63]:
for token in chain.stream({"context": docs}):
    print(token, end="|")

|

## Map-Reduce

In [66]:
from langchain_core.prompts import ChatPromptTemplate

map_prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

In [ ]:
#from langchain import hub

#We can also use the Prompt Hub to store and fetch prompts.
#This will work with your LangSmith API key.
# see prompt here: 

#map_prompt = hub.pull("rlm/map-prompt")

In [67]:
# Also available via the hub: `hub.pull("rlm/reduce-prompt")`
reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill it into a final, consolidated summary
of the main themes.
"""

reduce_prompt = ChatPromptTemplate([("human", reduce_template)])

In [68]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=0)

split_docs = text_splitter.split_documents(docs)
print(f"generated {len(split_docs)} documents")

generated 3 documents


In [69]:
# Invoke chain
result = chain.invoke({"context": [split_docs[0]]})
print(result)

The content appears to be a navigation guide for a website, likely related to GitHub or a similar platform. It includes sections such as "Troubleshooting," "API reference," and various guides like "Get started" and "How-to Guides." Additionally, there is an invitation to attend the "Interrupt: The Agent AI Conference by LangChain" in San Francisco on May 13 & 14. The navigation structure suggests a focus on learning resources, deployment information, and foundational concepts for users.


In [57]:
for token in chain.stream({"context": split_docs}):
    print(token, end="|")

The| document| provides| a| guide| on| troubleshooting| common| errors| encountered| while| using| the| Lang|Graph| platform|.| It| is| part| of| a| broader| resource| available| at| the| "|Lang|Graph| Platform|"| site|,| which| includes| sections| like| Home|,| Resources|,| and| Trou|b|leshooting|.

|Key| points| include|:

|-| **|Error| Reference|**:| The| page| lists| common| errors| with| specific| error| codes| such| as| `|GRAPH|_RE|CUR|SION|_LIMIT|`,| `|INVALID|_CON|CURRENT|_GRAPH|_UPDATE|`,| `|INVALID|_GRAPH|_NODE|_RETURN|_VALUE|`,| `|MULT|IPLE|_SUB|GR|AP|HS|`,| `|INVALID|_CHAT|_HISTORY|`,| and| `|INVALID|_LICENSE|`.

|-| **|Trou|b|leshooting| Guides|**:| These| guides| help| resolve| issues| related| to| the| Lang|Graph| platform|,| particularly| focusing| on| errors| specific| to| it|.

|-| **|Navigation|**:| The| document| includes| navigation| options| to| move| between| different| troubleshooting| topics| or| return| to| previous| sections|.

|The| content| is| part| of| a|